# NDVI Calculation with GeoProcessor

This notebook demonstrates how to calculate the Normalized Difference Vegetation Index (NDVI) using the GeoProcessor library. NDVI is a commonly used remote sensing index for assessing vegetation health and density.

## Setup

First, let's import the necessary modules from GeoProcessor.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Import GeoProcessor modules
import geoprocessor
from geoprocessor.raster import io as raster_io
from geoprocessor.raster import analysis
from geoprocessor.visualization import maps, plots

## Download Sample Landsat Data

For this example, we'll use Landsat 8 data from the AWS Open Data Registry. We'll need the red (Band 4) and near-infrared (Band 5) bands to calculate NDVI.

In [ ]:
import shutil
import requests

# Create data directory if it doesn't exist
data_dir = "../../data/raster"
os.makedirs(data_dir, exist_ok=True)

# Download red band (B4)
red_url = "https://landsat-pds.s3.amazonaws.com/c1/L8/153/075/LC08_L1TP_153075_20190515_20190515_01_RT/LC08_L1TP_153075_20190515_20190515_01_RT_B4.TIF"
red_path = os.path.join(data_dir, "landsat8_red.tif")

# Download near-infrared band (B5)
nir_url = "https://landsat-pds.s3.amazonaws.com/c1/L8/153/075/LC08_L1TP_153075_20190515_20190515_01_RT/LC08_L1TP_153075_20190515_20190515_01_RT_B5.TIF"
nir_path = os.path.join(data_dir, "landsat8_nir.tif")

# Download files if they don't exist
for url, path, band_name in [(red_url, red_path, "red"), (nir_url, nir_path, "NIR")]:
    if not os.path.exists(path):
        print(f"Downloading {band_name} band to {path}...")
        response = requests.get(url, stream=True)
        with open(path, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        print(f"{band_name} band download complete.")
    else:
        print(f"{band_name} band already exists at {path}")

## Load and Examine the Bands

Let's load the red and NIR bands and examine their properties.

In [ ]:
# Load the red and NIR bands
red_band = raster_io.read_raster(red_path)
nir_band = raster_io.read_raster(nir_path)

# Print basic information about the bands
print("Red band shape:", red_band.data.shape)
print("NIR band shape:", nir_band.data.shape)
print("\nRed band CRS:", red_band.crs)
print("NIR band CRS:", nir_band.crs)
print("\nRed band NoData value:", red_band.nodata)
print("NIR band NoData value:", nir_band.nodata)

## Visualize the Bands

Let's visualize the red and NIR bands to better understand the data.

In [ ]:
# Plot the bands side by side for comparison
fig = plots.plot_band_comparison(
    raster_list=[red_band, nir_band],
    labels=["Red Band (B4)", "NIR Band (B5)"],
    title="Landsat 8 Red and NIR Bands",
    figsize=(15, 8)
)

## Calculate NDVI

Now let's calculate NDVI using the formula:

$$NDVI = \frac{NIR - Red}{NIR + Red}$$

The GeoProcessor library provides a built-in function to calculate NDVI:

In [ ]:
# Set output path for NDVI result
ndvi_path = os.path.join(data_dir, "landsat8_ndvi.tif")

# Calculate NDVI
ndvi = analysis.calculate_ndvi(red_band, nir_band, output_path=ndvi_path)

print(f"NDVI calculation complete. Result saved to {ndvi_path}")

# Alternatively, we can calculate NDVI without saving to a file
ndvi = analysis.calculate_ndvi(red_band, nir_band)

## Visualize NDVI Results

Let's visualize the NDVI results using a colormap suitable for vegetation indices.

In [ ]:
# Create a static map of the NDVI result
fig = maps.create_static_map(
    raster=ndvi,
    title="Landsat 8 NDVI",
    cmap="RdYlGn",  # Red-Yellow-Green colormap (good for NDVI)
    vmin=-1,
    vmax=1
)

## Analyze NDVI Statistics

Let's calculate and analyze the statistics of our NDVI result.

In [ ]:
# Calculate statistics for the NDVI raster
ndvi_stats = analysis.calculate_statistics(ndvi)

# Print the statistics
print("NDVI Statistics:")
for key, value in ndvi_stats.items():
    print(f"  {key}: {value}")

## Create a Histogram of NDVI Values

Let's create a histogram to visualize the distribution of NDVI values.

In [ ]:
# Plot a histogram of NDVI values
fig = plots.plot_histogram(
    raster=ndvi,
    title="NDVI Distribution",
    xlabel="NDVI Value",
    bins=100,
    show_stats=True,
    color="green"
)

## Classify NDVI Values

Let's classify the NDVI values into different vegetation categories based on standard thresholds:

- < 0: Water, built-up areas, bare soil
- 0 - 0.2: Sparse vegetation
- 0.2 - 0.4: Moderate vegetation
- 0.4 - 0.6: Dense vegetation
- > 0.6: Very dense vegetation

In [ ]:
# Create a function to classify NDVI values
def classify_ndvi(ndvi_data):
    # Create a copy to avoid modifying the original data
    classified = np.zeros_like(ndvi_data)
    
    # Define thresholds and classes
    classified[ndvi_data < 0] = 1  # Water, built-up areas, bare soil
    classified[(ndvi_data >= 0) & (ndvi_data < 0.2)] = 2  # Sparse vegetation
    classified[(ndvi_data >= 0.2) & (ndvi_data < 0.4)] = 3  # Moderate vegetation
    classified[(ndvi_data >= 0.4) & (ndvi_data < 0.6)] = 4  # Dense vegetation
    classified[ndvi_data >= 0.6] = 5  # Very dense vegetation
    
    return classified

# Create a classified NDVI image
ndvi_classified = classify_ndvi(ndvi.data[0])

# Define a custom colormap for the classes
from matplotlib.colors import ListedColormap
cmap = ListedColormap(['darkblue', 'brown', 'yellow', 'lightgreen', 'darkgreen'])

# Plot the classified NDVI
plt.figure(figsize=(12, 10))
plt.imshow(ndvi_classified, cmap=cmap, vmin=1, vmax=5)

# Add a colorbar with labels
cbar = plt.colorbar(ticks=[1.4, 2.2, 3, 3.8, 4.6])
cbar.set_ticklabels(['Water/Built-up/Bare', 'Sparse Vegetation', 'Moderate Vegetation', 
                     'Dense Vegetation', 'Very Dense Vegetation'])

plt.title("Classified NDVI")
plt.axis('off')
plt.tight_layout()
plt.show()

## Calculate Vegetation Area

Let's calculate the area covered by different vegetation density classes. We'll need the pixel resolution to convert pixel counts to area.

In [ ]:
# Calculate pixel resolution in meters
pixel_width = abs(ndvi.transform[0])  # Width of a pixel in CRS units
pixel_height = abs(ndvi.transform[4])  # Height of a pixel in CRS units
pixel_area = pixel_width * pixel_height  # Area of a pixel in square CRS units

print(f"Pixel resolution: {pixel_width} x {pixel_height} units")
print(f"Pixel area: {pixel_area} square units")

# Calculate the area of each class in square kilometers
class_names = [
    "Water/Built-up/Bare",
    "Sparse Vegetation",
    "Moderate Vegetation",
    "Dense Vegetation",
    "Very Dense Vegetation"
]

class_areas = []
for i in range(1, 6):  # Classes 1 to 5
    class_pixel_count = np.sum(ndvi_classified == i)
    class_area = class_pixel_count * pixel_area / 1_000_000  # Convert to square kilometers
    class_areas.append(class_area)
    print(f"{class_names[i-1]}: {class_area:.2f} km²")

# Create a pie chart of vegetation class areas
plt.figure(figsize=(10, 8))
plt.pie(class_areas, labels=class_names, autopct='%1.1f%%', colors=['darkblue', 'brown', 'yellow', 'lightgreen', 'darkgreen'])
plt.title("Area by Vegetation Class")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.tight_layout()
plt.show()

## Save the Processed Results

Let's save the classified NDVI result as a GeoTIFF file.

In [ ]:
# Create a GeoRaster from the classified NDVI
from geoprocessor.core.datamodel import GeoRaster

# Expand dimensions to add the band dimension
ndvi_classified_expanded = np.expand_dims(ndvi_classified, 0)

# Create a GeoRaster object
classified_raster = GeoRaster(
    data=ndvi_classified_expanded,
    transform=ndvi.transform,
    crs=ndvi.crs,
    nodata=0,  # 0 is not used in our classification
    metadata={"description": "Classified NDVI"}
)

# Save to file
classified_path = os.path.join(data_dir, "landsat8_ndvi_classified.tif")
classified_raster.write(classified_path)

print(f"Classified NDVI saved to {classified_path}")

## Conclusion

In this notebook, we've demonstrated how to use the GeoProcessor library to:

1. Load and examine Landsat 8 red and NIR bands
2. Calculate NDVI using the built-in function
3. Visualize and analyze NDVI results
4. Classify NDVI values into vegetation density categories
5. Calculate and visualize the area covered by each vegetation class
6. Save the processed results as GeoTIFF files

NDVI is a powerful tool for vegetation analysis and environmental monitoring. The GeoProcessor library makes it easy to calculate, analyze, and visualize NDVI from multispectral satellite imagery.